# below code is for Market basket analysis

In [1]:
import pandas as pd
jj_raw_data=pd.read_csv("jj_2021_to_june_2024_raw_data_for_MBA.csv")

In [2]:
jj_raw_data

,DATE,ONLINE_PRODUCT_NAME,ORDER_ID,ONLINE_PRODUCT_ID,CATEGORY,USER_ID,ONLINE_REVENUE
0,2021-01-01,Dark Grey Drawstring Sweatpants,996730,51633,Others,65db07722dfa602de5adbf67,1120.00
1,2021-01-01,Black Tape Detail Trackpants,996774,59353,Others,65a80404be2448966e073f75,999.00
2,2021-01-02,Navy Blue Check Full Sleeves Shirt,997748,54952,Others,65a02b60be2448966ed46921,1120.00
3,2021-01-02,Black Polo Neck T-shirt,997774,59124,Others,65db077c2dfa602de5adc13d,1039.00
4,2021-01-02,Black Polo Neck T-shirt,997930,59124,Others,65db077d2dfa602de5adc17d,1039.00
...,...,...,...,...,...,...,...
1320727,2024-06-24,Brown Premium Leather Card Holder,3479956,413359,Wallets,65a02c3ebe2448966ed492d2,1218.00
1320728,2024-06-25,Black Premium Leather Card Holder,3483499,413358,Wallets,667b6ce4b37d8b54ee4c5b29,1118.00
1320729,2024-06-25,Black Leather Wallet,3482515,414301,Wallets,667b6ce0b37d8b54ee4c59dc,1038.51
1320730,2024-06-26,Black Premium Leather Card Holder,3485217,413358,Wallets,667cbe60f696d74b6026caaa,1118.00


In [3]:
# first remove for duplicates
jj_raw_data.drop_duplicates(inplace=True)

In [4]:
jj_raw_data

,DATE,ONLINE_PRODUCT_NAME,ORDER_ID,ONLINE_PRODUCT_ID,CATEGORY,USER_ID,ONLINE_REVENUE
0,2021-01-01,Dark Grey Drawstring Sweatpants,996730,51633,Others,65db07722dfa602de5adbf67,1120.00
1,2021-01-01,Black Tape Detail Trackpants,996774,59353,Others,65a80404be2448966e073f75,999.00
2,2021-01-02,Navy Blue Check Full Sleeves Shirt,997748,54952,Others,65a02b60be2448966ed46921,1120.00
3,2021-01-02,Black Polo Neck T-shirt,997774,59124,Others,65db077c2dfa602de5adc13d,1039.00
4,2021-01-02,Black Polo Neck T-shirt,997930,59124,Others,65db077d2dfa602de5adc17d,1039.00
...,...,...,...,...,...,...,...
1320727,2024-06-24,Brown Premium Leather Card Holder,3479956,413359,Wallets,65a02c3ebe2448966ed492d2,1218.00
1320728,2024-06-25,Black Premium Leather Card Holder,3483499,413358,Wallets,667b6ce4b37d8b54ee4c5b29,1118.00
1320729,2024-06-25,Black Leather Wallet,3482515,414301,Wallets,667b6ce0b37d8b54ee4c59dc,1038.51
1320730,2024-06-26,Black Premium Leather Card Holder,3485217,413358,Wallets,667cbe60f696d74b6026caaa,1118.00


### We create the transactional data

In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
# Order_id wise transactional data
def create_transactional_data_mlxtend(df):
    # Load the data
    data = df
    
    # Group the data by ORDER_ID and collect all online_product_name into a list per order
    grouped = data.groupby('ORDER_ID')['ONLINE_PRODUCT_NAME'].apply(list)
    
    # Extract the order IDs to use as index later
    order_ids = grouped.index.tolist()
    
    # Convert grouped data to list for TransactionEncoder
    grouped_list = grouped.tolist()
    
    te = TransactionEncoder()
    
    te_ary = te.fit(grouped_list).transform(grouped_list)
    
    # Convert the array into a DataFrame with item names as columns
    transaction_data = pd.DataFrame(te_ary, columns=te.columns_, index=order_ids)
    
    return transaction_data


ModuleNotFoundError: No module named 'mlxtend'

#### to map the categories of antecedent and consequent

In [46]:
def map_categories(rules_df, category_df):
    # Create a dictionary from category_df mapping products to categories
    product_to_category = dict(zip(category_df['ONLINE_PRODUCT_NAME'], category_df['CATEGORY']))
    
    # Helper function to extract category for a set of products
    def get_categories(product_set):
        # Remove the set notation and split by ', ' to handle multiple items
        products = str(product_set).replace("frozenset({", "").replace("})", "").replace("'", "").split(', ')
        # Find the category of each product, handling missing products by returning 'Unknown'
        categories = {product_to_category.get(product, 'Unknown') for product in products}
        return ', '.join(categories)  # Return categories as a comma-separated string if multiple

    # Apply the helper function to antecedents and consequents
    rules_df['antecedent_category'] = rules_df['antecedents'].apply(get_categories)
    rules_df['consequent_category'] = rules_df['consequents'].apply(get_categories)
    
    rules_df[['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift','antecedent_category','consequent_category']]
    
    return rules_df


In [48]:
jj_transactional_data_product_name=create_transactional_data_mlxtend(jj_raw_data)
jj_transactional_data_product_name

,BLACK KNIT CREW NECK T-SHIRT,BLACK LOGO PRINT KNIT CREW NECK T-SHIRT,Black All Over Print Full Sleeves Shirt,Black Camo Print Sneakers,Black Colourblocked Crew Neck T-shirt,Black Drawstring Sweatshorts,Black Front Zip Crew Neck T-shirt,Black Logo Print Crew Neck T-shirt,Black Pride Rainbow Crew Neck T-shirt,Black Striped Crew Neck T-shirt,...,x Pok mon Black Thunderbolt Crew Neck T-shirt,x Pok mon Green Ombre Crew Neck T-shirt,x Pok mon White Colourblocked Crew Neck T-shirt,x Pok mon White Printed Crew Neck T-shirt,x Pokémon Black Thunderbolt Crew Neck T-shirt,x Pokémon Green Ombre Crew Neck T-shirt,x Pokémon White Colourblocked Crew Neck T-shirt,x Pokémon White Printed Crew Neck T-shirt,x Snoopy White Snoopy Print Crew Neck T-shirt,‘PROTECT’ Dark Blue Low Rise Tim Slim Fit Jeans
996678,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
996680,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
996684,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
996685,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
996686,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493650,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3493659,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3493668,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3493674,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [50]:
from mlxtend.frequent_patterns import apriori, association_rules,fpgrowth
fpgrowth_jj = fpgrowth(jj_transactional_data_product_name, min_support=0.00001, use_colnames=True)
rules_jj = association_rules(fpgrowth_jj, metric="confidence", min_threshold=0.01)
rules_jj

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Black Jacquard Polo Neck T-shirt),(Blue Polo T-Shirt),0.000567,0.000417,0.000012,0.021552,51.687291,0.000012,1.021600,0.981210
1,(Blue Polo T-Shirt),(Black Jacquard Polo Neck T-shirt),0.000417,0.000567,0.000012,0.029326,51.687291,0.000012,1.029627,0.981062
2,(Blue Polo T-Shirt),(Green Logo Print Crew Neck T-shirt),0.000417,0.005727,0.000015,0.035191,6.144211,0.000012,1.030538,0.837594
3,(Black Logo Print Crew Neck T-Shirt),(Blue Polo T-Shirt),0.001078,0.000417,0.000011,0.010204,24.472350,0.000011,1.009888,0.960173
4,(Blue Polo T-Shirt),(Black Logo Print Crew Neck T-Shirt),0.000417,0.001078,0.000011,0.026393,24.472350,0.000011,1.026001,0.959538
...,...,...,...,...,...,...,...,...,...,...
23681,(Light Brown Graphic Printed Oversized T-shirt),"(White Graphic Printed Oversized T-shirt, Blac...",0.000241,0.000093,0.000026,0.106599,1147.085025,0.000026,1.119214,0.999369
23682,(Red Printed Cotton Boxers),(Dark Grey Printed Cotton Boxers),0.000044,0.000024,0.000012,0.277778,11358.569444,0.000012,1.384582,0.999956
23683,(Dark Grey Printed Cotton Boxers),(Red Printed Cotton Boxers),0.000024,0.000044,0.000012,0.500000,11358.569444,0.000012,1.999912,0.999936
23684,(Pack Of 2 Dark Grey & Green Trunks),(Pack Of 2 Navy Blue &amp; Black Trunks),0.000070,0.000187,0.000015,0.210526,1125.307190,0.000015,1.266430,0.999181


In [52]:
rules_jj_with_cat=map_categories(rules_jj,jj_raw_data)

In [54]:
rules_jj_with_cat.to_csv("rules_jj_with_cat.csv",index=False)

In [56]:
def clean_frozenset(df, columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: next(iter(eval(x))) if pd.notna(x) else x)
    return df

In [58]:
rules_jj_with_cat=pd.read_csv("rules_jj_with_cat.csv")

In [60]:
rules_jj_with_cat

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_category,consequent_category
0,frozenset({'Black Jacquard Polo Neck T-shirt'}),frozenset({'Blue Polo T-Shirt'}),0.000567,0.000417,0.000012,0.021552,51.687291,0.000012,1.021600,0.981210,T-Shirts,T-Shirts
1,frozenset({'Blue Polo T-Shirt'}),frozenset({'Black Jacquard Polo Neck T-shirt'}),0.000417,0.000567,0.000012,0.029326,51.687291,0.000012,1.029627,0.981062,T-Shirts,T-Shirts
2,frozenset({'Blue Polo T-Shirt'}),frozenset({'Green Logo Print Crew Neck T-shirt'}),0.000417,0.005727,0.000015,0.035191,6.144211,0.000012,1.030538,0.837594,T-Shirts,T-Shirts
3,frozenset({'Black Logo Print Crew Neck T-Shirt'}),frozenset({'Blue Polo T-Shirt'}),0.001078,0.000417,0.000011,0.010204,24.472350,0.000011,1.009888,0.960173,T-Shirts,T-Shirts
4,frozenset({'Blue Polo T-Shirt'}),frozenset({'Black Logo Print Crew Neck T-Shirt'}),0.000417,0.001078,0.000011,0.026393,24.472350,0.000011,1.026001,0.959538,T-Shirts,T-Shirts
...,...,...,...,...,...,...,...,...,...,...,...,...
23681,frozenset({'Light Brown Graphic Printed Oversi...,frozenset({'White Graphic Printed Oversized T-...,0.000241,0.000093,0.000026,0.106599,1147.085025,0.000026,1.119214,0.999369,T-Shirts,T-Shirts
23682,frozenset({'Red Printed Cotton Boxers'}),frozenset({'Dark Grey Printed Cotton Boxers'}),0.000044,0.000024,0.000012,0.277778,11358.569444,0.000012,1.384582,0.999956,Boxers,Boxers
23683,frozenset({'Dark Grey Printed Cotton Boxers'}),frozenset({'Red Printed Cotton Boxers'}),0.000024,0.000044,0.000012,0.500000,11358.569444,0.000012,1.999912,0.999936,Boxers,Boxers
23684,frozenset({'Pack Of 2 Dark Grey & Green Trunks'}),frozenset({'Pack Of 2 Navy Blue &amp; Black Tr...,0.000070,0.000187,0.000015,0.210526,1125.307190,0.000015,1.266430,0.999181,Trunks,Trunks


In [62]:
rules_jj_with_cat=clean_frozenset(rules_jj_with_cat,["antecedents","consequents"])

In [64]:
rules_jj_with_cat.to_csv("rules_jj_with_cat.csv",index=False)

In [66]:
rules_jj_with_cat

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_category,consequent_category
0,Black Jacquard Polo Neck T-shirt,Blue Polo T-Shirt,0.000567,0.000417,0.000012,0.021552,51.687291,0.000012,1.021600,0.981210,T-Shirts,T-Shirts
1,Blue Polo T-Shirt,Black Jacquard Polo Neck T-shirt,0.000417,0.000567,0.000012,0.029326,51.687291,0.000012,1.029627,0.981062,T-Shirts,T-Shirts
2,Blue Polo T-Shirt,Green Logo Print Crew Neck T-shirt,0.000417,0.005727,0.000015,0.035191,6.144211,0.000012,1.030538,0.837594,T-Shirts,T-Shirts
3,Black Logo Print Crew Neck T-Shirt,Blue Polo T-Shirt,0.001078,0.000417,0.000011,0.010204,24.472350,0.000011,1.009888,0.960173,T-Shirts,T-Shirts
4,Blue Polo T-Shirt,Black Logo Print Crew Neck T-Shirt,0.000417,0.001078,0.000011,0.026393,24.472350,0.000011,1.026001,0.959538,T-Shirts,T-Shirts
...,...,...,...,...,...,...,...,...,...,...,...,...
23681,Light Brown Graphic Printed Oversized T-shirt,White Graphic Printed Oversized T-shirt,0.000241,0.000093,0.000026,0.106599,1147.085025,0.000026,1.119214,0.999369,T-Shirts,T-Shirts
23682,Red Printed Cotton Boxers,Dark Grey Printed Cotton Boxers,0.000044,0.000024,0.000012,0.277778,11358.569444,0.000012,1.384582,0.999956,Boxers,Boxers
23683,Dark Grey Printed Cotton Boxers,Red Printed Cotton Boxers,0.000024,0.000044,0.000012,0.500000,11358.569444,0.000012,1.999912,0.999936,Boxers,Boxers
23684,Pack Of 2 Dark Grey & Green Trunks,Pack Of 2 Navy Blue &amp; Black Trunks,0.000070,0.000187,0.000015,0.210526,1125.307190,0.000015,1.266430,0.999181,Trunks,Trunks


In [35]:
# above is the final rule set that we could come up with